# content 

- explain constraint programming
- explain basic setup
- show basic example from google
- CP-SAT solver
- CP-Solver


# Constraint programming

In constraint programming or constraint optimization the goal is to idendtify a feasible solution, where the problem can be modeled in terms of arbitary constraints.

## Install OR-Tools

Installing OR-Tools is straight forward using conda:

```conda install -c conda-forge ortools-python```

or simply pip

```python -m pip install --upgrade --user ortools```

for more information see [Google OR-Tools Overview](https://developers.google.com/optimization/install)

## Example count enemy soldiers

We want to defeat the army of our opponent and therefore we have send out scouts to count the number of soldiers. Lets denote the number of soldiers by $n$.

Because counting the soldiers one by one would be cumbersome, we asked our scouts to simply count the number of soldiers in the first row while they were marching through different landscapes.

Hence the scouts reported:

- scout 1: "At the bridge there were 13 soldiers in first row"
- scout 2: "At the crossing there were 19 soldiers in the first row"
- scout 3: "In the open field there were 37 soldiers in the first row"

So lets translate this into math: Scout 1 says that the number of soldiers is divisible by 13 and scout 2 (respk. scout 3) says that the number is divisible by 19 (respk. 37). And this can written as [congruences](https://en.wikipedia.org/wiki/Modular_arithmetic)

$$
\begin{array}{lll}
n & \equiv 0 & \mod (13) \\
n & \equiv 0 & \mod (19) \\
n & \equiv 0 & \mod (37) 
\end{array}
$$

In addition we somehow know that the number of soldiers of the opponents army can not exceed 10000 soldiers.


*remark:* For simplicity of this example we assume that each row has the same number of soldiers. However if there would be less soldiers in the last row, then the left hand side of the above congruence would be excatly this number.


### background info

The example is well known since the 3 century and is a simple application of the [chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) and requires only some simple math to solve it directly.
But we like to point out that the use of congrunences in a LP for example is not that straight forward.

## general steps

1. instantiate the model and the solver
1. declare variables 
1. declare constraints
1. (optional) define objective
1. apply solver

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
# instantiate the model and the solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [ ]:
# declare variable (together with bounds)
army = model.NewIntVar(1,100000, 'army')

In [ ]:
# declare constraints
model.AddModuloEquality(0, army, 13)
model.AddModuloEquality(0, army, 19)
model.AddModuloEquality(0, army, 37)

<ortools.sat.python.cp_model.Constraint>

In [ ]:
# apply solver
status = solver.Solve(model)
# print solver status
print('solver status:' + str(status == cp_model.OPTIMAL))
print('solution, soldiers in enemy army:' + str(solver.Value(army)))

solver status:True
solution, soldiers in enemy army:9139


# Basic operations in OR tools - Model manipulations

c.f. [github OR-tools](https://github.com/google/or-tools/blob/stable/ortools/sat/docs/model.md) for more details

## changing bounds 

Previously we have assumed that the army can not have more than 10000 soldiers. Lets see what happends if we set this bound much higher.

The following snippet shows, how to change a variable bound

In [ ]:
new_value = 100000
army.Proto().domain[:] = []
army.Proto().domain.extend(cp_model.Domain(1, new_value).FlattenedIntervals())
army.Proto()

name: "army"
domain: 1
domain: 100000

add a solution printer

In [ ]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [ ]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(army)

In [ ]:
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model)

In [ ]:
status = solver.Solve(model)

In [ ]:
print('Status = %s' % solver.StatusName(status))
print('Number of solutions found: %i' % solution_printer.solution_count())


Status = OPTIMAL
Number of solutions found: 0
